In [1]:
import pandas as pd

# todo: test how accurate the gender prediciton and age prediction are. Save the number and report it to Will.
df = pd.read_excel('../static/demographic-others-labels.xlsx', sheet_name='Final Values')
keep_names = ['Filename', 'Age', 'Gender']
df_select = df[keep_names]
df_select.to_pickle('../static/gender_age_2k_gt.pkl')


In [10]:
df = pd.read_pickle('../static/gender_age_2k_gt.pkl')

In [23]:
15. / 2222.

0.0067506750675067504

In [53]:
import numpy as np
data = np.load('predicted_age_gen.npz')

In [54]:
age_pred = data['age']

In [55]:
gender_pred = data['gender']

In [56]:
gender = gender_pred[:, 1]

In [57]:
gender = [round(i) for i in gender]

In [51]:
sum(df['Gender'].values == gender)

1731

In [52]:
1731/ 2222.

0.779027902790279

In [58]:
sum(df['Gender'].values == gender)

2084

In [59]:
2084 / 2222.

0.9378937893789379

In [60]:
from os import listdir
from os.path import isfile, join
mypath = '/home/amanda/Documents/VC and Entrepreneur Faces Project/New_data/crunchbase/0/0a0/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [64]:
mypath = '/home/amanda/Documents/VC and Entrepreneur Faces Project/New_data/crunchbase/0/0a0/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f)) and f[0] != '.']

In [66]:
import skimage.io
import skimage.transform
import os
import cv2
import tensorflow as tf
import numpy as np
import skimage
import skimage.io
import skimage.transform
import math
import time
import pandas as pd
import pickle




# Right now, we only deal with crunchbase, new data. It's saved in
# '/home/amanda/Documents/VC and Entrepreneur Faces Project/New_data/crunchbase/'
# A sample folder is '/0/'


# These parameters are for file list generation.
raw_data_root_dir = '/home/amanda/Documents/VC and Entrepreneur Faces Project/New_data/crunchbase/'
raw_data_dir_postfix = '0/'

raw_data_dir = raw_data_root_dir + raw_data_dir_postfix
crop_with_mask_root_dir = '/home/amanda/Documents/cropped_new_crunchbase/'
crop_with_mask_dir = crop_with_mask_root_dir + raw_data_dir_postfix

end_with_key_word = 'g'  # png or jpeg or jpg.


# These parameters are for impression prediction network.
default_device = '/gpu:0'
num_hidden_neurons = 256

vgg_mean = [103.939, 116.779, 123.68]
weights_path = '/home/amanda/Documents/vgg_model/vgg16_weights.npz'
batch_size = 100
feat_save_dir = '/home/amanda/Documents/feat_folder/'
model_save_dir = '/home/amanda/Documents/vgg_model/2k_summer_analysis/'


In [68]:
    def fetch_file(path_to_folder, flag, key_word):
        '''
        fetch_files() requires three arguments: pathToFolder, flag and keyWord

        flag must be 'start_with' or 'end_with'
        keyWord is a string to search the file's name

        Be careful, the keyWord is case sensitive and must be exact

        Example: fetch_file('/Documents/Photos/','end_with','.jpg')

        returns: _pathToFiles and _fileNames
        '''

        _path_to_files = []
        _file_names = []

        for dir_path, dir_names, file_names in os.walk(path_to_folder):
            if flag == 'end_with':
                raw_file = [item for item in file_names if item.endswith(key_word)]
                selected_file = [item for item in raw_file if not item.startswith('._')]
                selected_file = list(set(selected_file))
                _file_names.extend(selected_file)

                selected_path = [os.path.join(dir_path, item) for item in selected_file]
                _path_to_files.extend(selected_path)

            elif flag == 'start_with':
                raw_file = [item for item in file_names if item.startswith(key_word)]
                selected_file = [item[2:] if item.startswith('._') else item for item in raw_file]
                selected_file = list(set(selected_file))
                _file_names.extend(selected_file)

                selected_path = [os.path.join(dir_path, item) for item in selected_file]
                _path_to_files.extend(selected_path)

            else:
                print fetch_file.__doc__
                break

            # try to remove empty entries if none of the required files are in the directory
            try:
                _path_to_files.remove('')
                _file_names.remove('')
            except ValueError:
                pass

            # warn if nothing was found in the given path
            if selected_file == []:
                print('No files with given parameters were found in {}\n'.format(dir_path))

        print('{} files are found in searched folders.'.format(len(_file_names)))
        return _path_to_files, _file_names

In [69]:
path_to_files, file_names = fetch_file(raw_data_dir, 'end_with', end_with_key_word)

No files with given parameters were found in /home/amanda/Documents/VC and Entrepreneur Faces Project/New_data/crunchbase/0/

3668 files are found in searched folders.


In [70]:
len(path_to_files)

3668

In [82]:
from __future__ import division
num_files = len(file_names)
batch_size = 100
num_batches = int(np.ceil(num_files / batch_size))

In [83]:
num_batches

37

In [87]:
valid_file_lst = []
for i in range(num_batches):
#     batch_filenames = file_names[i * batch_size: (i + 1) * batch_size]
    if i == num_batches - 1:
        batch_filenames = file_names[i * batch_size:]
    else:
        batch_filenames = file_names[i * batch_size: (i + 1) * batch_size]
    valid_file_lst += batch_filenames

In [88]:
len(valid_file_lst)

3668

In [89]:
for i, (j, k) in enumerate(zip(file_names, path_to_files)):
    print i, j, k

0 0f0b6480-3569-fa60-6a1b-c1808711afb8_CB.jpg /home/amanda/Documents/VC and Entrepreneur Faces Project/New_data/crunchbase/0/0f0/0f0b6480-3569-fa60-6a1b-c1808711afb8_CB.jpg
1 0f08a7a1-5aae-40eb-9c97-819e492377e4_CB.jpg /home/amanda/Documents/VC and Entrepreneur Faces Project/New_data/crunchbase/0/0f0/0f08a7a1-5aae-40eb-9c97-819e492377e4_CB.jpg
2 0f008ff5-f552-4ad8-b35d-ea1f9d57b0c4_CB.jpg /home/amanda/Documents/VC and Entrepreneur Faces Project/New_data/crunchbase/0/0f0/0f008ff5-f552-4ad8-b35d-ea1f9d57b0c4_CB.jpg
3 0f074dbb-ac39-159c-c62e-e0e5027e45ed_CB.png /home/amanda/Documents/VC and Entrepreneur Faces Project/New_data/crunchbase/0/0f0/0f074dbb-ac39-159c-c62e-e0e5027e45ed_CB.png
4 0f08b1ab-e345-400c-a462-44824829a3e6_CB.jpg /home/amanda/Documents/VC and Entrepreneur Faces Project/New_data/crunchbase/0/0f0/0f08b1ab-e345-400c-a462-44824829a3e6_CB.jpg
5 0f06a699-732f-1b1a-dd3f-57e4aeddaa48_CB.png /home/amanda/Documents/VC and Entrepreneur Faces Project/New_data/crunchbase/0/0f0/0f06a6

985 0d0e37f9-36ae-c2ff-a5f3-e89f085cccc8_CB.png /home/amanda/Documents/VC and Entrepreneur Faces Project/New_data/crunchbase/0/0d0/0d0e37f9-36ae-c2ff-a5f3-e89f085cccc8_CB.png
986 0d004421-3151-cd12-d69e-f7fd7ed7f1bb_CB.jpg /home/amanda/Documents/VC and Entrepreneur Faces Project/New_data/crunchbase/0/0d0/0d004421-3151-cd12-d69e-f7fd7ed7f1bb_CB.jpg
987 0d0311eb-1030-a239-c1a2-36f741cb6b39_CB.jpg /home/amanda/Documents/VC and Entrepreneur Faces Project/New_data/crunchbase/0/0d0/0d0311eb-1030-a239-c1a2-36f741cb6b39_CB.jpg
988 0d0bc034-4cb8-9f28-a639-a9d28c1500d8_CB.jpg /home/amanda/Documents/VC and Entrepreneur Faces Project/New_data/crunchbase/0/0d0/0d0bc034-4cb8-9f28-a639-a9d28c1500d8_CB.jpg
989 0d092ef9-ecdb-3388-900f-b15633a97774_CB.jpg /home/amanda/Documents/VC and Entrepreneur Faces Project/New_data/crunchbase/0/0d0/0d092ef9-ecdb-3388-900f-b15633a97774_CB.jpg
990 0d098150-e691-a038-9b59-33db1270c68e_CB.png /home/amanda/Documents/VC and Entrepreneur Faces Project/New_data/crunchbase/

1942 0db20e42-f4ac-afa1-d5ef-a06be9af7add_CB.jpg /home/amanda/Documents/VC and Entrepreneur Faces Project/New_data/crunchbase/0/0db/0db20e42-f4ac-afa1-d5ef-a06be9af7add_CB.jpg
1943 0db1e881-1f44-70c7-47df-de579c279387_CB.jpg /home/amanda/Documents/VC and Entrepreneur Faces Project/New_data/crunchbase/0/0db/0db1e881-1f44-70c7-47df-de579c279387_CB.jpg
1944 0db53d04-18e6-3e39-15d8-07d84bc2aef4_CB.jpg /home/amanda/Documents/VC and Entrepreneur Faces Project/New_data/crunchbase/0/0db/0db53d04-18e6-3e39-15d8-07d84bc2aef4_CB.jpg
1945 0db15b11-8d03-9b43-df8e-a0128c0217b0_CB.jpg /home/amanda/Documents/VC and Entrepreneur Faces Project/New_data/crunchbase/0/0db/0db15b11-8d03-9b43-df8e-a0128c0217b0_CB.jpg
1946 0db336de-7dca-d494-d65f-0e31ca6a3e2c_CB.jpg /home/amanda/Documents/VC and Entrepreneur Faces Project/New_data/crunchbase/0/0db/0db336de-7dca-d494-d65f-0e31ca6a3e2c_CB.jpg
1947 0db37274-f363-241b-d23f-743034b68fec_CB.png /home/amanda/Documents/VC and Entrepreneur Faces Project/New_data/crunc

2961 0e651707-f9f6-640f-c23b-879f47d7c19d_CB.jpg /home/amanda/Documents/VC and Entrepreneur Faces Project/New_data/crunchbase/0/0e6/0e651707-f9f6-640f-c23b-879f47d7c19d_CB.jpg
2962 0e605cb7-289b-49ad-8e0e-968511b5851d_CB.jpg /home/amanda/Documents/VC and Entrepreneur Faces Project/New_data/crunchbase/0/0e6/0e605cb7-289b-49ad-8e0e-968511b5851d_CB.jpg
2963 0e6280c2-50e3-429d-beb2-7860a5f4c5c6_CB.jpg /home/amanda/Documents/VC and Entrepreneur Faces Project/New_data/crunchbase/0/0e6/0e6280c2-50e3-429d-beb2-7860a5f4c5c6_CB.jpg
2964 0e61967c-d2b8-ba36-882a-c00708a03f75_CB.png /home/amanda/Documents/VC and Entrepreneur Faces Project/New_data/crunchbase/0/0e6/0e61967c-d2b8-ba36-882a-c00708a03f75_CB.png
2965 0e62bcf7-8db1-784f-1c65-a06851930fdc_CB.png /home/amanda/Documents/VC and Entrepreneur Faces Project/New_data/crunchbase/0/0e6/0e62bcf7-8db1-784f-1c65-a06851930fdc_CB.png
2966 0e657830-7972-3292-f682-c5ccde1dd6d4_CB.jpg /home/amanda/Documents/VC and Entrepreneur Faces Project/New_data/crunc